In [23]:
from Transformer.Config import Config
from Transformer.Model import Transformer
import torch.nn as nn
import torch
from Seq2SeqModel.Seq2SeqModel import Seq2SeqModel
import pandas as pd
from transformers import AutoTokenizer

# CSV 파일에서 데이터 불러오기
df_train = pd.read_csv("Datatset3/train.csv")
df_val = pd.read_csv("Datatset3/val.csv")
tokenizer= AutoTokenizer.from_pretrained("Datatset3/tokenizer")

def str_to_list(input_string):
    cleaned_string = input_string.strip("[]").replace(" ", "")

    number_strings = cleaned_string.split(",")
    number_list = [int(num) for num in number_strings]
    return number_list

# 데이터 추출
X_train = df_train["X_train"].values.tolist()
X_train=[str_to_list(i) for i in X_train]
y_train = df_train["y_train"].values.tolist()
y_train=[str_to_list(i) for i in y_train]


X_val = df_val["X_val"].values.tolist()
X_val=[str_to_list(i) for i in X_val]
y_val = df_val["y_val"].values.tolist()
y_val=[str_to_list(i) for i in y_val]



In [26]:
from collections import Counter
def to_unk_dset(dataset_2d,vocab):
    # 데이터셋에서 모든 센텐스를 하나의 1차원 리스트로 펼칩니다.
    flat_sentences = [token for sentence in dataset_2d for token in sentence]

    # 단어 빈도를 계산합니다.
    word_counts = Counter(flat_sentences)

    # 등장 빈도가 낮은 단어를 찾아 <unk>로 대체하고 vocab에서 제거합니다.
    min_word_frequency = 2  # 등장 빈도가 이 값보다 작으면 <unk>로 대체
    filtered_sentences = []
    for sentence in dataset_2d:
        filtered_sentence = [token if word_counts[token] >= min_word_frequency else '<unk>' for token in sentence]
        filtered_sentences.append(filtered_sentence)

    # vocab에서 <unk>를 제거합니다.
    vocab = list(set(flat_sentences))
    if '<unk>' in vocab:
        vocab.remove('<unk>')

    # 처리된 데이터셋과 vocab을 사용하여 모델을 학습하거나 다른 작업을 수행할 수 있습니다.


In [ ]:
tokenizer.vocab

In [ ]:

device = torch.device('mps')

print(f'Using {device}')

tokenizer= AutoTokenizer.from_pretrained("Datatset/tokenizer")
config=Config(3)
model=Transformer(config)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-11)
batchsize=64
print("Set")
module=Seq2SeqModel( model,tokenizer,optimizer,loss_fn,
                    X_train,y_train,X_val,y_val)


In [2]:
module.train_main(3,5)

Epoch: 1/3   Batch: 5/5   cost: 0.0103371: |████████████████    |80.00%


        Running_Loss: 0.01 VAL_ACC: 0 

|                    |0.00%


KeyboardInterrupt: 

In [ ]:
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(X_train[0])))
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y_train[0])))
print(X_train[0])
print(y_train[0])

<bos> ▁Calibration is▁about to check the▁value▁range your▁device▁delivers.▁Please▁move▁axis %1 %2 on your▁device to the maximum position. Press▁any▁button on the▁device or▁click on the'Next 'button to continue with the▁next▁step.
<bos> Le calibrage va vérifier la plage de valeurs que votre matériel produit. Veuillez déplacer l'axe %1 %2 de votre périphérique à la position maximale. Appuyez sur n'importe quel bouton du périphérique ou sur le bouton « & #160; Suivant & #160; » pour la prochaine étape.</s>
[59514, 34378, 226, 5783, 32, 200, 12, 3647, 4, 1223, 1628, 117, 4923, 23608, 3, 1789, 2942, 20059, 301, 548, 301, 331, 30, 117, 4923, 12, 4, 1528, 668, 3, 5734, 212, 9319, 30, 4, 4923, 57, 5487, 30, 4, 6, 32712, 25, 7243, 1160, 12, 621, 42, 4, 1156, 3009, 3, 0]
[59514, 60, 7418, 5244, 8234, 740, 4993, 8, 6471, 5, 2218, 29, 193, 2220, 742, 3, 4366, 14237, 14, 6, 16600, 301, 548, 301, 331, 5, 193, 24275, 17, 8, 668, 6142, 3, 33640, 36, 81, 6, 5411, 2709, 9376, 22, 24275, 59, 36, 19, 9376

In [ ]:
module.model.eval()
i_e=torch.tensor(X_train[0][:-1])
out=torch.tensor(module.seq_to_seq_process(i_e))
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(i_e)))
print(out,out.shape)
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(out)))

seq_padd:  128
seq_padd:  128
<bos> ▁Calibration is▁about to check the▁value▁range your▁device▁delivers.▁Please▁move▁axis %1 %2 on your▁device to the maximum position. Press▁any▁button on the▁device or▁click on the'Next 'button to continue with the▁next▁step.
tensor([17833,  2332,  2332,  2332, 17154, 17154, 51385, 51385, 51385, 17154,
        17154, 17154, 51385, 51385, 25337, 17154, 17154, 40291, 40291, 21398,
        33386,  4875, 51385, 51385, 51385, 51385, 30212, 30212, 30212, 30212,
        30212, 51385, 51385, 51385, 51385, 51385, 51385, 51385, 51385, 51385,
        50510, 50510, 50510, 51385, 51385, 51385, 51671, 50983, 21398, 21398,
        21398, 50510, 49137, 49137, 50983, 51385, 51385, 51385, 25337, 25337,
        25337, 51385, 51385,   860,   860,   860, 47255, 47255, 51671, 51671,
        51671, 51671, 51385, 51385, 51385, 51385, 51385, 51385, 40269, 40269,
        51385, 51385, 51385, 45835, 45835, 47255, 47255, 47255, 51385, 51385,
        49137,   924, 47255, 47255, 51